In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import random
import warnings

warnings.filterwarnings('ignore')

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

def execute_query(query_code, menu_data):
    try:
        # Ensure proper formatting with indentation
        formatted_query_code = '\n'.join(
            '    ' + line if line.strip() else line for line in query_code.split('\n')
        )

        # Print the formatted query code for debugging
        print("Formatted Query Code:")
        print(formatted_query_code)

        # Define a safe context with local variables
        local_vars = {'menu_data': menu_data, 'query': query, 'item': None, 'result': None}

        exec(f"""
def run_query(menu_data):
    item = None
    {formatted_query_code}
    return item
""", {"__builtins__": None}, local_vars)

        # Call the query function
        result = local_vars['run_query'](menu_data)

        # Check if item is None
        if result is None:
            print("Debug Info: No matching item found in menu_data.")
        else:
            print("Debug Info: Found item:", result)

    except Exception as e:
        print(f"Error executing query: {e}")
        result = "No result found."

    return result



# Generate response
def response(sentence):
    intent_name = predict_intent(sentence)
    intent_config = next((i for i in intents_config if i['intent'] == intent_name), None)

    if intent_config and 'query' in intent_config:
        query_code = intent_config['query']
        # Execute the query
        result = execute_query(query_code, menu_data)
        return result, intent_name
    else:
        return random.choice(response_for_intent.get(intent_name, ["Sorry, I didn't understand the request."])), intent_name

# Test the system
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break
    bot_response, typ = response(query)
    print(f'Geek: {bot_response} -- TYPE: {typ}')
    print()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0588 - loss: 2.2021
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.1791
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.1501
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2941 - loss: 2.1559
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3529 - loss: 2.1303
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2941 - loss: 2.1565
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3529 - loss: 2.0942
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5294 - loss: 2.0791
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1176 - loss: 2.0940
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.0796
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4706 - loss: 1.9876
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4118 - loss

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import random
import warnings
import time  # Import the time module

warnings.filterwarnings('ignore')

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# Generate response (updated to only return intent)
def response(sentence):
    intent_name = predict_intent(sentence)
    return intent_name

# Test the system with time tracking
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break

    # Start time tracking
    start_time = time.time()

    # Get the predicted intent
    predicted_intent = response(query)

    # End time tracking
    end_time = time.time()

    # Calculate the time taken
    time_taken = end_time - start_time

    # Print the predicted intent and time taken
    print(f'Predicted Intent: {predicted_intent}')
    print(f'Time taken: {time_taken:.4f} seconds')
    print()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.1932
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2353 - loss: 2.1789
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1176 - loss: 2.1734
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5294 - loss: 2.1497
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4118 - loss: 2.1384
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4706 - loss: 2.1182
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4118 - loss: 2.1242
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4118 - loss: 2.0940
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4706 - loss: 2.0755
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4706 - loss: 2.0174
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4706 - loss: 2.0620
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4706 - 

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import pickle

# Load intent configuration
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/intent1.json', 'r') as f:
    intents_config = json.load(f)['intents']

# Load menu data
with open('/Users/yash/MagilHub/Project 1/OfflineChatBot/OfflineChatBotCode/mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

for intent in intents_config:
    intent_name = intent['intent']

    if intent_name not in unique_intents:
        unique_intents.append(intent_name)

    for text in intent['text']:
        text_input.append(text)
        intents.append(intent_name)

    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    for response in intent['responses']:
        response_for_intent[intent_name].append(response)

    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('query', '')

tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Save the model and tokenizer to a pickle file
with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump({'model': model, 'tokenizer': tokenizer, 'intent_to_index': intent_to_index, 'index_to_intent': index_to_intent}, f)

# Function to load the model and tokenizer from pickle
def load_model_and_tokenizer(pickle_file):
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
        return data['model'], data['tokenizer'], data['intent_to_index'], data['index_to_intent']

# Load model and tokenizer
model, tokenizer, intent_to_index, index_to_intent = load_model_and_tokenizer('chatbot_model.pkl')

# Predict intent
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# Generate response (updated to only return intent)
def response(sentence):
    intent_name = predict_intent(sentence)
    return intent_name

# Test the system
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break

    # Get the predicted intent
    predicted_intent = response(query)

    # Print the predicted intent
    print(f'Predicted Intent: {predicted_intent}')
    print()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1765 - loss: 2.1978
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4706 - loss: 2.1807
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3529 - loss: 2.1671
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4118 - loss: 2.1432
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2941 - loss: 2.1473
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4118 - loss: 2.1266
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4118 - loss: 2.1107
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3529 - loss: 2.0835
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3529 - loss: 2.0849
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3529 - loss: 2.0803
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3529 - loss: 2.0496
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2941 - loss